### 【問題1】クロスバリデーション
```
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。
具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。
```

### 【問題2】グリッドサーチ
```
これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。
グリッドサーチをパイプラインの中に組み込みましょう。
```

### 【問題3】Kernelからの調査
```
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。
```

### 【問題4】高い汎化性能のモデル
```
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。
今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。
```